In [192]:
import pandas as pd # Importação da biblioteca pandas para leitura dos dados
import numpy as np # Importação da biblioteca np para manipulação de dados



In [193]:
dados_ano=pd.read_csv("dados.txt") # Leitura dos dados de geração e carga de 1 ano

In [194]:
dados_ano

,data,diaDoAno,mes,dia,hora,vento,radiacao,demanda
0,01-Jan-2017,1,1,1,0,2.7,0.0,1931.58
1,01-Jan-2017,1,1,1,1,2.5,0.0,1871.10
2,01-Jan-2017,1,1,1,2,2.1,0.0,1823.22
3,01-Jan-2017,1,1,1,3,2.3,0.0,1748.88
4,01-Jan-2017,1,1,1,4,2.8,0.0,1693.44
...,...,...,...,...,...,...,...,...
8755,31-Dec-2017,365,12,31,19,5.5,0.0,2331.00
8756,31-Dec-2017,365,12,31,20,4.3,0.0,2269.26
8757,31-Dec-2017,365,12,31,21,3.6,0.0,2150.82
8758,31-Dec-2017,365,12,31,22,4.3,0.0,2034.90


In [195]:
def lerArquivoIEEE(nomeArquivo):
    dadosGerais=[]
    dadosBarras=[]
    dadosRamos=[]
    try:
        file = open(nomeArquivo,'r')
    except OSError:
        return False
    else:
        for linha in file:
            dadosGerais.append(linha[:-1])
        file.close()
    sbase=float(dadosGerais[0][31:36])
    linha=2
    while True:
        apenasNumeros=dadosGerais[linha][:5]+dadosGerais[linha][17:]
        dadosBarras.append(apenasNumeros.split())
        linha+=1
        if dadosGerais[linha][:4]=='-999' or linha>=len(dadosGerais):
            break
    linha+=2
    while True:
        dadosRamos.append(dadosGerais[linha].split())
        linha+=1
        if dadosGerais[linha][:4]=='-999' or linha>=len(dadosGerais):
            break
    nb=len(dadosBarras)
    dadosBarras=np.array(dadosBarras,dtype=np.float64)
    dadosRamos=np.array(dadosRamos,dtype=np.float64)
    dfBarras=pd.DataFrame(dadosBarras,columns=['bus','area','zone','type','voltage',
											'angle','pl','ql','pg','qg',
											'base_voltage','desired_volts','q_max','q_min',
											'g_shunt','b_shunt','control_bus'])
    dfRamos=pd.DataFrame(dadosRamos,columns=['from','to','area','zone','circuit','type','resistance',
											'reactance','charging_b','rating_1','rating_2','rating_3',
											'control_bus','side','tap','trans_phase','tap_min','tap_max',
											'step_size','volt_min','volt_max'])
    dfRamos['tap']=dfRamos['tap'].replace(0.0, 1.0)
    dfRamos['from']=[np.where(dfBarras['bus'] == i)[0][0] for i in dfRamos['from']]
    dfRamos['to']=[np.where(dfBarras['bus'] == i)[0][0] for i in dfRamos['to']]
    dfBarras=dfBarras.drop('bus', axis=1)
    return (nb, #Numero de barras
        sbase, # S base
        dfBarras,
        dfRamos) 

In [196]:
def geracao_solar(dados_ano):
    # Defiições do painel fotovoltaico utilizado
    n_PV = 0.173
    n_conv = 0.9
    area_painel = 1.66
    numero_paineis = 200
    # Geração fotovoltaica
    Pot_PV = dados_ano.radiacao * n_PV * n_conv * area_painel * numero_paineis
    Pot_PV = np.array(Pot_PV)
    return Pot_PV

In [197]:
def geracao_eolica(dados_ano):
    # Definições para ajuste  de velocidade do vento
    H0 = 2
    H = 15
    Fator_rugosidade = 0.23
    vento_corrigido = dados_ano.vento * (H / H0) ** Fator_rugosidade

    # Definições do aerogerador utilizado
    Pot_max_GE = 2400 #kW
    Vc = 3.5
    Vf = 25
    Vr = 9.5
    numero_aerogeradores = 50
    
    # Geração eólica
    Pot_GE = np.zeros(len(vento_corrigido))
    index = set(dados_ano.index[vento_corrigido >= Vc].tolist()) & set(dados_ano.index[vento_corrigido <= Vr].tolist())
    Pot_GE[list(index)] = Pot_max_GE * ((vento_corrigido[list(index)] - Vc) / (Vr - Vc)) * numero_aerogeradores
    index = set(dados_ano.index[vento_corrigido >= Vr].tolist()) & set(dados_ano.index[vento_corrigido <= Vf].tolist())
    Pot_GE[list(index)] = Pot_max_GE * numero_aerogeradores
    return Pot_GE

In [198]:
def cal_net_load(dados_ano):
    # Carga da rede
    carga = np.array(dados_ano.demanda)
    net_load = np.zeros(len(dados_ano.hora))
    net_load = geracao_eolica(dados_ano) + geracao_solar(dados_ano) - carga
    return net_load

In [199]:
def funcionamento_bateria(net_load, BESS_energy, soc_min, soc_max, soc_int, dados_ano): # Soe = State of Energy
    soe= np.zeros(len(dados_ano.hora))
    soe_min = soc_min * BESS_energy
    soe_max = soc_max * BESS_energy
    soe_int = soc_int * BESS_energy
    soe[0] = soe_int
    local_loss = 0
    lpsp = 0
    for i in range(len(dados_ano.hora)-1): 
        if soe[i] >= soe_min and soe[i] <= soe_max:
            soe[i+1] = soe[i] + (net_load[i]+net_load[i+1])/2
            if soe[i+1] > soe_max:
                soe[i+1] = soe_max
            elif soe[i+1] < soe_min:
                soe[i+1] = soe_min
                local_loss = local_loss + (net_load[i]+net_load[i+1])/2 - (soe[i]-soe_min)
        lpsp=lpsp+abs(local_loss)
    lpsl=lpsp/(sum(dados_ano.demanda)*len(dados_ano.demanda))
    return lpsl

In [200]:
def enxame_inicial(N_pop, limites, dim):
    BESS_energy = np.zeros((N_pop, dim))
    for i in range(N_pop):
        for j in range(dim):
            BESS_energy[i,j] = np.random.uniform(limites[0], limites[1])
    return BESS_energy

In [201]:
#Avalia o custo da posicao avaliada
def calcula_custo_bess(net_load, BESS_energy, soc_min, soc_max, soc_int, dados_ano, lpsl_max):
    lpsl = funcionamento_bateria(net_load, BESS_energy, soc_min, soc_max, soc_int, dados_ano)
    if lpsl > lpsl_max:
        custo_bess = BESS_energy+lpsl*20000000
    else:
        custo_bess = BESS_energy
    return custo_bess

In [202]:
#Input dos Dados
soc_min=0.1
soc_max=0.9
soc_int=0.5
lpsl_max = 0.01

tol = 1e-6
kmax = 10
K_run=10
N_pop=40

w = 0.5                   
c1 = 2.0                  
c2 = 2.0                  

dim = 1                     
max_iter = 100             
limites = (0, 20000)
iterMax = 20

p_val = np.zeros(N_pop)

p_best_val = np.zeros(N_pop)        
g_best_val = np.zeros(1)          

p_best = np.zeros((N_pop,dim))      
g_best = np.zeros(dim)     

BESS_energy = enxame_inicial(N_pop, limites, dim)
net_load = cal_net_load(dados_ano)

for ii in range(N_pop):
    p_best_val[ii] = calcula_custo_bess(net_load, BESS_energy[ii], soc_min, soc_max, soc_int, dados_ano, lpsl_max)
    p_best [ii] = BESS_energy[ii]

ind = np.argmin(p_best_val)                      
g_best_val = np.copy(p_best_val[ind]) 
g_best = np.copy(p_best[ind,:])  

v = np.zeros((N_pop,dim))  

x=BESS_energy

for iter in range(iterMax):
    rp = np.random.rand(dim,N_pop)         
    rg = np.random.rand(dim,N_pop)           
    
    y=g_best.reshape(1,len(g_best))
    z=p_best.reshape(dim,len(p_best))

    v_global = np.multiply(((x-y)),rg.transpose())*c2*(-1.0)    
    v_local = np.multiply((z.T- x),rp.T)*c1          
    
    v = w*v + (v_local + v_global)       

    for ii in range(N_pop):
        for jj in range(dim):
            if v[ii,jj] > x_max - x[ii,jj]:
                v[ii,jj] = x_max - x[ii,jj]
                continue
            elif v[ii,jj] < x_min - x[ii,jj]:
                v[ii,jj] = x_min - x[ii,jj]
    x = x + v

    x=np.round(x).astype(int)

    for ii in range(N_pop):
        p_val[ii] = calcula_custo_bess(net_load, BESS_energy[ii], soc_min, soc_max, soc_int, dados_ano, lpsl_max)
        if p_val[ii] < p_best_val[ii]:
            p_best_val[ii] = p_val[ii]
            p_best [ii] = x[ii]

    ind = np.argmin(p_best_val)                      
    g_best_val = np.copy(p_best_val[ind]) 
    g_best = np.copy(p_best[ind,:])  


print(g_best_val)
print(g_best)

11187.036760883391
[11187.03676088]
